In [1]:
from open3d import geometry
from open3d import utility
from open3d import io
from open3d import visualization
from PyGEL3D import gel
import numpy as np
from ipywidgets import FloatProgress
from IPython.display import display
import os
import glob

In [2]:
def create_mesh_line_set(mesh, color=[1,0,0]):
    line_set = geometry.create_line_set_from_triangle_mesh(mesh)
    colors = [color for _ in range(len(line_set.lines))]
    line_set.colors = utility.Vector3dVector(colors)
    return line_set

def load_meshes(folderdir, filename):
    filedirs = glob.glob(f'{folderdir}*.ply')
    meshes = []
    for fdir in filedirs:
        if filename == fdir[:-5]:
            meshes.append(read_mesh(fdir))
    return meshes

def read_mesh(filedir):
    assert os.path.exists(filedir), 'File Directory does not exist'
    assert filedir.endswith('.ply'), 'Mesh File must be OBJ or PLY'
    print('PLY Mesh File Loaded')
    mesh = io.read_triangle_mesh(filedir)
    return mesh

def gelread_mesh(filedir):
    assert os.path.exists(filedir), 'File Directory does not exist'
    assert filedir.endswith('.ply') or filedir.endswith('.obj'), 'Mesh File must be OBJ or PLY'
    if filedir.endswith('.ply'):
        print('PLY Mesh File Loaded')
        mesh = gel.ply_load(filedir)
    elif filedir.endswith('.obj'):
        print('OBJ Mesh File Loaded')
        mesh = gel.obj_load(filedir)
    return mesh

def write_mesh(filedir, mesh):
    assert not os.path.exists(filedir), 'File Directory already exists'
    assert filedir.endswith('.ply') or filedir.endswith('.obj'), 'Mesh File must be OBJ or PLY'
    io.write_triangle_mesh(filedir, mesh)
    print('Mesh successfully written')

def get_centroids(mesh):
    faces = np.asarray(mesh.faces())
    centroids = np.zeros((len(faces),3))
    for face in faces:
        centroids[face] = np.array(mesh.centre(face))
    return centroids

def get_polygons(mesh):
    vertices = np.asarray(mesh.vertices())
    polygons = []
    for vertex in vertices:
        polygons.append(list(mesh.circulate_vertex(vertex, mode='f')))
    return polygons

def polygon_triangles(mesh):
    triangles = []
    vertices = get_centroids(mesh)
    polygons = get_polygons(mesh)
    f = FloatProgress(min=0, max=len(polygons)-1)
    display(f)
    count = 0
    for polygon in polygons:
        center = np.array([0.0,0.0,0.0])
        for idx in polygon:
            center += vertices[idx]
        center /= len(polygon)
        center_idx = len(vertices)
        vertices = np.append(vertices, [center], axis=0)
        triangles.append([center_idx, polygon[-1], polygon[0]])
        for idx in range(0, len(polygon)-1):
            triangles.append([center_idx, polygon[idx], polygon[idx + 1]])
        count += 1
        if count%int(len(polygons)/1000) == 0:
            f.value = count
    return vertices, np.asarray(triangles)

def generate_marble(filedir, marbledir, write=True):
    assert not os.path.exists(marbledir), 'Output File Directory already exists'
    assert marbledir.endswith('.ply') or marbledir.endswith('.obj'), 'Mesh File must be OBJ or PLY'
    mesh = gelread_mesh(filedir)

    print('Generating Marble Mesh')
    vertices, triangles = polygon_triangles(mesh)
    marble_mesh = geometry.TriangleMesh()
    marble_mesh.vertices = utility.Vector3dVector(vertices)
    marble_mesh.triangles = utility.Vector3iVector(triangles)
    marble_mesh.compute_triangle_normals()
    marble_mesh.compute_vertex_normals()
    print('Marble Mesh Complete')
    if write:
        write_mesh(marbledir, marble_mesh)
    return marble_mesh

In [3]:
folderdir = '/Users/nickf/Dropbox/Nicky T-Shirt Project/Subject Scans/Male Scans/Nicholas Ferrara/NF_SCAN_V3/'
filedirs = glob.glob(f'{folderdir}*_M1.ply')
for filedir in filedirs:
    marbledir = f'{filedir[:-5]}2.ply'
    mesh = generate_marble(filedir, marbledir, write=True)
#filedir = f'{folderdir}NFMarkers_V2_M1.ply'
#marbledir = f'{folderdir}NFMarkers_V2_M2.ply'
#mesh = generate_marble(filedir, marbledir, write=True)

AssertionError: Output File Directory already exists

In [13]:
mesh = generate_marble("/Users/nickf/Desktop/Face_Scan_1_Marble_3.ply", "/Users/nickf/Desktop/Face_Scan_1_Marble_4.ply", write=True)

PLY Mesh File Loaded
Generating Marble Mesh


FloatProgress(value=0.0, max=388693.0)

Marble Mesh Complete
Mesh successfully written


In [14]:
visualization.draw_geometries([mesh])

In [19]:
folderdir = '/Users/nickf/Desktop/Custom-Fit/Data/Subject Meshes/'
filenames = [f'{folderdir}nf0_100.ply', f'{folderdir}nf0_200.ply', f'{folderdir}nf0_300.ply', f'{folderdir}nf0_400.ply']
#meshes = load_meshes(folderdir, filename)
geometries = []
for i, file in enumerate(filenames):
    mesh = read_mesh(file)
    mesh.compute_vertex_normals()
    geometries.append(mesh)
    geometries.append(create_mesh_line_set(mesh, color=[i,i+2,i-1]))

PLY Mesh File Loaded
PLY Mesh File Loaded
PLY Mesh File Loaded
PLY Mesh File Loaded


In [ ]:
visualization.draw_geometries(geometries)

In [15]:
marbledir = '/Users/nickf/Desktop/Custom-Fit/Data/Subject Meshes/nf0_100.ply'
mesh = read_mesh(marbledir)
mesh.compute_vertex_normals()
visualization.draw_geometries([mesh])

PLY Mesh File Loaded
